<a href="https://colab.research.google.com/github/groda/big_data/blob/master/Hadoop_Setting_up_a_Single_Node_Cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/groda/big_data"><div><img src="https://github.com/groda/big_data/blob/master/logo_bdb.png?raw=true" align=right width="90"></div></a>

# HDFS and MapReduce on a single-node Hadoop cluster
<br>
<br>

We're going to set up a single-node cluster (following the instructions on https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html) and show how to run simple HDFS and MapReduce commands.

After downloading the software, it is necessary to carry out some preliminary steps like setting environment variables, generating SSH keys, etc.). We grouped all these steps under "Prologue".

Once done with the prologue, we are able to start a single-node Hadoop cluster on the current virtual machine.

We are going to run some test HDFS commands and MapReduce jobs on the Hadoop cluster.

Finally, the cluster will be shut down.


## Prologue

### Check the available Java version
 Apache Hadoop 3.3 and upper supports Java 8 and Java 11 (runtime only). See: https://cwiki.apache.org/confluence/display/HADOOP/Hadoop+Java+Versions


Check if Java version is one of `8`, `11`

In [ ]:
!java -version

openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode, sharing)


In [ ]:
%%bash
JAVA_MAJOR_VERSION=$(java -version 2>&1 | grep -m1 -Po '(\d+\.)+\d+' | cut -d '.' -f1)
if [[ $JAVA_MAJOR_VERSION -eq 8 || $JAVA_MAJOR_VERSION -eq 11 ]]
 then 
 echo "Java version is one of 8, 11 ✓" 
 fi

Java version is one of 8, 11 ✓


Find the variable for the environment variable `JAVA_HOME`

Find the path for the environment variable `JAVA_HOME`

In [ ]:
!readlink -f $(which java)

/usr/lib/jvm/java-11-openjdk-amd64/bin/java


Extract JAVA_HOME from the Java path by removing the `bin/java` part in the end

In [ ]:
%%bash
JAVA_HOME=$(readlink -f $(which java) | sed 's/\/bin\/java$//')
echo $JAVA_HOME

/usr/lib/jvm/java-11-openjdk-amd64


We're going to use `JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64`.

Use `%env%` to set the variable for the current notebook session.

In [ ]:
#%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

### Download core Hadoop 
Download the latest stable version of the core Hadoop distribution from one of the download mirrors locations https://www.apache.org/dyn/closer.cgi/hadoop/common/.

**Note** with the option `--no-clobber`, `wget` will not download the file if it already exists.

In [ ]:
!wget --no-clobber https://dlcdn.apache.org/hadoop/common/stable/hadoop-3.3.4.tar.gz

File ‘hadoop-3.3.4.tar.gz’ already there; not retrieving.



Uncompress archive

In [ ]:
!tar xzf hadoop-3.3.4.tar.gz 

### Verify the downloaded file 

(see https://www.apache.org/dyn/closer.cgi/hadoop/common/)

Download sha512 file

In [ ]:
! wget --no-clobber https://dlcdn.apache.org/hadoop/common/stable/hadoop-3.3.4.tar.gz.sha512

File ‘hadoop-3.3.4.tar.gz.sha512’ already there; not retrieving.



Compare

In [ ]:
%%bash
A=$(sha512sum hadoop-3.3.4.tar.gz | cut - -d' ' -f1)
B=$(cut hadoop-3.3.4.tar.gz.sha512 -d' ' -f4)
printf "%s\n%s\n" $A $B
[[ $A == $B ]] && echo "True"

ca5e12625679ca95b8fd7bb7babc2a8dcb2605979b901df9ad137178718821097b67555115fafc6dbf6bb32b61864ccb6786dbc555e589694a22bf69147780b4
ca5e12625679ca95b8fd7bb7babc2a8dcb2605979b901df9ad137178718821097b67555115fafc6dbf6bb32b61864ccb6786dbc555e589694a22bf69147780b4
True


### Configure `PATH`

Add the Hadoop folder to the `PATH` environment variable


In [ ]:
!echo $PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


In [ ]:
import os
#os.environ['HADOOP_HOME'] = os.path.join(os.environ['HOME'], 'hadoop-3.3.4')
os.environ['HADOOP_HOME'] = os.path.join('/content', 'hadoop-3.3.4')
os.environ['PATH'] = ':'.join([os.path.join(os.environ['HADOOP_HOME'], 'bin'), os.environ['PATH']])
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [ ]:
import os
print(os.environ)

environ({'NV_LIBCUBLAS_DEV_VERSION': '11.4.1.1043-1', 'NV_CUDA_COMPAT_PACKAGE': 'cuda-compat-11-2', 'NV_CUDNN_PACKAGE_DEV': 'libcudnn8-dev=8.1.1.33-1+cuda11.2', 'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64', 'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.8.4-1+cuda11.2', 'TCLLIBPATH': '/usr/share/tcltk/tcllib1.19', 'CLOUDSDK_PYTHON': 'python3', 'LANG': 'en_US.UTF-8', 'NV_LIBNPP_DEV_PACKAGE': 'libnpp-dev-11-2=11.3.2.152-1', 'HOSTNAME': '93ad6cb45e12', 'OLDPWD': '/', 'CLOUDSDK_CONFIG': '/content/.config', 'NV_LIBNPP_VERSION': '11.3.2.152-1', 'NV_NVPROF_DEV_PACKAGE': 'cuda-nvprof-11-2=11.2.152-1', 'NVIDIA_VISIBLE_DEVICES': 'all', 'NV_NVPROF_VERSION': '11.2.152-1', 'NV_LIBCUSPARSE_VERSION': '11.4.1.1152-1', 'DATALAB_SETTINGS_OVERRIDES': '{"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["--ip=172.28.0.2"],"debugAdapterMultiplexerPath":"/usr/local/bin/dap_multiplexer","enableLsp":true}', 'NV_LIBCUBLAS_DEV_PACKAGE': 'libcublas-dev-11-2=11.4.1.

In [ ]:
!echo $PATH

/content/hadoop-3.3.4/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin


### Configure `core-site.xml` and `hdfs-site.xml`

Edit the file `etc/hadoop/core-site.xml` and `etc/hadoop/hdfs-site.xml` to configure pseudo-distributed operation.

**`etc/hadoop/core-site.xml`**
```
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://localhost:9000</value>
    </property>
</configuration>
```

**`etc/hadoop/hdfs-site.xml`**
```
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>
```

In [ ]:
%%bash
echo -e "<configuration> \n\
    <property> \n\
        <name>fs.defaultFS</name> \n\
        <value>hdfs://localhost:9000</value> \n\
    </property> \n\
</configuration>" >hadoop-3.3.4/etc/hadoop/core-site.xml

echo -e "<configuration> \n\
    <property> \n\
        <name>dfs.replication</name> \n\
        <value>1</value> \n\
    </property> \n\
</configuration>" >hadoop-3.3.4/etc/hadoop/hdfs-site.xml

Check

In [ ]:
cat hadoop-3.3.4/etc/hadoop/hdfs-site.xml

<configuration> 
    <property> 
        <name>dfs.replication</name> 
        <value>1</value> 
    </property> 
</configuration>


### Set environment variables

Add the following lines to the Hadoop configuration script `hadoop-env.sh`(the script is in `hadoop-3.3.4/sbin`).
```
export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64
export HDFS_NAMENODE_USER=root
export HDFS_DATANODE_USER=root
export HDFS_SECONDARYNAMENODE_USER=root
export YARN_RESOURCEMANAGER_USER=root
export YARN_NODEMANAGER_USER=root
```

In [ ]:
%%bash
echo -e "export HDFS_NAMENODE_USER=root \n\
export HDFS_DATANODE_USER=root \n\
export HDFS_SECONDARYNAMENODE_USER=root \n\
export YARN_RESOURCEMANAGER_USER=root \n\
export YARN_NODEMANAGER_USER=root" >> hadoop-3.3.4/etc/hadoop/hadoop-env.sh

### Setup localhost access via SSH key

We are going to allow passphraseless access to `localhost` with a secure key.

SSH must be installed and sshd must be running to use the Hadoop scripts that manage remote Hadoop daemons.



#### Install `openssh` and start server

I'm not sure why we need the option `StrictHostKeyChecking no`. This option tells the `ssh` server to allow key authentication only from known hosts, in particular it prevents a host from authenticating with key if the key has changed. I guess this option is needed since a new ssh key is generated every time one runs this notebook.

Alternatively, one could just delete the file `~/.ssh/known_hosts` or else use `ssh-keygen -R hostname` to remove all keys belonging to hostname from the `known_hosts` file (see for instance [How to remove strict RSA key checking in SSH and what's the problem here?](https://serverfault.com/questions/6233/how-to-remove-strict-rsa-key-checking-in-ssh-and-whats-the-problem-here) or [Remove key from known_hosts](https://superuser.com/questions/30087/remove-key-from-known-hosts)). The option `ssh-keygen -R hostname` seems the most elegant and I'm going to try it out some day.


In [ ]:
%%bash
apt-get install openssh-server
echo 'StrictHostKeyChecking no' >> /etc/ssh/ssh_config
/etc/init.d/ssh restart

Reading package lists...
Building dependency tree...
Reading state information...
openssh-server is already the newest version (1:7.6p1-4ubuntu0.7).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
 * Restarting OpenBSD Secure Shell server sshd
   ...done.


#### Generate key
Generate SSH key that does not require a password. 

The private key is contained in the file `id_rsa` located in the folder `~/.ssh`.

The public key is added to the file `~/.ssh/authorized_keys` in order to allow authentication with that key.

In [ ]:
%%bash
rm /root/.ssh/id_rsa
ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
chmod 0600 ~/.ssh/authorized_keys

Generating public/private rsa key pair.
Your identification has been saved in /root/.ssh/id_rsa.
Your public key has been saved in /root/.ssh/id_rsa.pub.
The key fingerprint is:
SHA256:rtKGkhiJdO+m+xVvQByZtYUziVkEEaEGB+kvTi4RQrs root@93ad6cb45e12
The key's randomart image is:
+---[RSA 2048]----+
|   oo. *&+o.     |
| . .o o=.=o      |
|. o  o o .o      |
|.+ o. .          |
|+.+ o  oS        |
|+E o o .+        |
| o=.oo ..o       |
|..oooo+..        |
|  .+=+.          |
+----[SHA256]-----+


#### Check SSH connection to localhost

The following command should output "hi!" if the connection works.

In [ ]:
!ssh localhost "echo hi!"

hi!


## Launch a single-node Hadoop cluster

### Initialize the namenode

In [ ]:
!hdfs namenode -format -nonInteractive

namenode is running as process 24037.  Stop it first and ensure /tmp/hadoop-root-namenode.pid file is empty before retry.


In [ ]:
!echo -e "export JAVA_HOME=$(readlink -f $(which java) | sed 's/\/bin\/java$//') \n\
export HDFS_NAMENODE_USER=root \n\
export HDFS_DATANODE_USER=root \n\
export HDFS_SECONDARYNAMENODE_USER=root \n\
export YARN_RESOURCEMANAGER_USER=root \n\
export YARN_NODEMANAGER_USER=root" >> hadoop-3.3.4/etc/hadoop/hadoop-env.sh

### Start cluster

In [ ]:
!$HADOOP_HOME/sbin/start-dfs.sh

Starting namenodes on [localhost]
localhost: namenode is running as process 24037.  Stop it first and ensure /tmp/hadoop-root-namenode.pid file is empty before retry.
Starting datanodes
localhost: datanode is running as process 24169.  Stop it first and ensure /tmp/hadoop-root-datanode.pid file is empty before retry.
Starting secondary namenodes [93ad6cb45e12]
93ad6cb45e12: secondarynamenode is running as process 24411.  Stop it first and ensure /tmp/hadoop-root-secondarynamenode.pid file is empty before retry.


## Run some simple `hdfs` commands

In [ ]:
%%bash
# create directory "my_dir" in HDFS home 
hdfs dfs -mkdir /user
hdfs dfs -mkdir /user/root # this is the "home" of user root on HDFS
hdfs dfs -mkdir my_dir

# upload file mnist_test.csv to my_dir
hdfs dfs -put /content/sample_data/mnist_test.csv my_dir/

# show contents of directory my_dir
hdfs dfs -ls -h my_dir


Found 1 items
-rw-r--r--   1 root supergroup     17.4 M 2022-10-08 15:43 my_dir/mnist_test.csv


mkdir: `/user': File exists
mkdir: `/user/root': File exists
mkdir: `my_dir': File exists
put: `my_dir/mnist_test.csv': File exists


## Run some simple MrReduce jobs
We're going to use the [streaming](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html) library. 
WIth this utility any executable can be used as the mapper and/or the reducer. 

#### Simplest MapReduce job

We are going to use the Unix commands `cat` as mapper and `wc` as reducer so we don't need to code anything. The result will show a line with three values: the counts of lines, words, and characters in the input file(s).

Input folder is `/user/my_user/my_dir/`, output folder `/user/my_user/output`.

**Note**: the output folder should not exist because it is created by Hadoop (this is in acordance with Hadoop's principle of not overwriting data).

Now run the MapReduce job

In [ ]:
%%bash

hdfs dfs -rm -r output_simplest || hdfs namenode -format -nonInteractive
mapred streaming \
  -input my_dir \
  -output output_simplest \
  -mapper /bin/cat \
  -reducer /usr/bin/wc

Deleted output_simplest


2022-10-08 17:51:44,941 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-10-08 17:51:45,074 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-10-08 17:51:45,074 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-10-08 17:51:45,095 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-10-08 17:51:45,539 INFO mapred.FileInputFormat: Total input files to process : 1
2022-10-08 17:51:45,569 INFO mapreduce.JobSubmitter: number of splits:1
2022-10-08 17:51:45,865 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1420042903_0001
2022-10-08 17:51:45,865 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-10-08 17:51:46,106 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2022-10-08 17:51:46,107 INFO mapreduce.Job: Running job: job_local1420042903_0001
2022-10-08 17:51:46,114 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2022-10

If the `output` directory contains the empty file `_SUCCESS`, this means that the job was successful. 

Check the output of the MapReduce job.

In [ ]:
!hdfs dfs -cat output_simplest/part-00000

  10000   10000 18299443	


The number of words is in this case equal to the number of lines because there are no word separators (empty spaces) in the file, so each line is a word.

#### Another MapReduce example: filter a log file

We're going to use a Linux logfile and look for the string `sshd` in a given position. 

The mapper `mapper.py` filters the file for the given string `sshd` at field 4. 

The job has no reducer (option `-reducer NONE`). Note that without a reducer the sorting and shuffling phase after the map phase is skipped.


Download the logfile `Linux_2k.log`:

In [ ]:
!wget --no-clobber https://raw.githubusercontent.com/logpai/loghub/master/Linux/Linux_2k.log

File ‘Linux_2k.log’ already there; not retrieving.



In [ ]:
%%bash
hdfs dfs -mkdir input || true
hdfs dfs -put Linux_2k.log input/ || true

mkdir: `input': File exists
put: `input/Linux_2k.log': File exists


Define the mapper

In [ ]:
%%writefile mapper.py
#!/usr/bin/env python
import sys

for line in sys.stdin:
    # split the line into words
    line = line.strip()
    fields = line.split()
    if (len(fields)>=5 and fields[4].startswith('sshd')):
      print(line)


Overwriting mapper.py


Test the script (after setting the correct permissions)

In [ ]:
!chmod 700 mapper.py

Look at the first 10 lines

In [ ]:
!head -10 Linux_2k.log

Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4 
Jun 15 02:04:59 combo sshd(pam_unix)[20882]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20884]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20883]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20885]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd

Test the mapper in the shell (not using MapReduce):

In [ ]:
!head -100 Linux_2k.log| ./mapper.py 

Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4
Jun 15 02:04:59 combo sshd(pam_unix)[20882]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20884]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20883]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20885]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(p

Now run the MapReduce job on the pseudo-cluster

In [ ]:
%%bash

hdfs dfs -rm -r output_filter 

mapred streaming \
  -file mapper.py \
  -input input \
  -output output_filter \
  -mapper mapper.py \
  -reducer NONE 
  

Deleted output_filter
packageJobJar: [mapper.py] [] /tmp/streamjob7054732551868862576.jar tmpDir=null


2022-10-08 17:52:03,227 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
2022-10-08 17:52:04,618 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-10-08 17:52:04,776 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-10-08 17:52:04,776 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-10-08 17:52:04,795 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-10-08 17:52:05,145 INFO mapred.FileInputFormat: Total input files to process : 1
2022-10-08 17:52:05,173 INFO mapreduce.JobSubmitter: number of splits:1
2022-10-08 17:52:05,468 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1202847949_0001
2022-10-08 17:52:05,469 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-10-08 17:52:05,818 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/job_local12

Check the result

In [ ]:
!hdfs dfs -ls output_filter

Found 2 items
-rw-r--r--   1 root supergroup          0 2022-10-08 17:52 output_filter/_SUCCESS
-rw-r--r--   1 root supergroup      85436 2022-10-08 17:52 output_filter/part-00000


In [ ]:
!hdfs dfs -cat output_filter/part-00000 |head 

Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4	
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown	
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4	
Jun 15 02:04:59 combo sshd(pam_unix)[20882]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo sshd(pam_unix)[20884]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo sshd(pam_unix)[20883]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo sshd(pam_unix)[20885]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo

### Aggregate data with MapReduce

Following the example in [Hadoop Streaming/Aggregate package](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html#Hadoop_Aggregate_Package)

In [ ]:
%%writefile myAggregatorForKeyCount.py
#!/usr/local/bin/python
import sys

def generateLongCountToken(id):
    return "LongValueSum:" + id + "\t" + "1"

def main(argv):
    line = sys.stdin.readline()
    try:
        while line:
            line = line[:-1]
            fields = line.split()
            s = fields[4].split('[')[0]
            print(generateLongCountToken(s))
            line = sys.stdin.readline()
    except "end of file":
        return None

if __name__ == "__main__":
     main(sys.argv)

Overwriting myAggregatorForKeyCount.py


Set permissions

In [ ]:
!chmod 700 myAggregatorForKeyCount.py

Test the mapper 

In [ ]:
!head -20 Linux_2k.log| ./myAggregatorForKeyCount.py

LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:su(pam_unix)	1
LongValueSum:su(pam_unix)	1
LongValueSum:logrotate:	1
LongValueSum:su(pam_unix)	1
LongValueSum:su(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1


Run the MapReduce job

In [ ]:
%%bash

hdfs dfs -rm -r output_aggregate 

mapred streaming \
  -file mapper.py \
  -file myAggregatorForKeyCount.py \
  -input input \
  -output output_aggregate \
  -mapper myAggregatorForKeyCount.py \
  -reducer aggregate  
  

Deleted output_aggregate
packageJobJar: [mapper.py, myAggregatorForKeyCount.py] [] /tmp/streamjob12656127282969925127.jar tmpDir=null


2022-10-08 17:52:18,059 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
2022-10-08 17:52:19,464 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-10-08 17:52:19,625 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-10-08 17:52:19,625 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2022-10-08 17:52:19,644 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2022-10-08 17:52:20,047 INFO mapred.FileInputFormat: Total input files to process : 1
2022-10-08 17:52:20,073 INFO mapreduce.JobSubmitter: number of splits:1
2022-10-08 17:52:20,367 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1869278852_0001
2022-10-08 17:52:20,367 INFO mapreduce.JobSubmitter: Executing with tokens: []
2022-10-08 17:52:20,663 INFO mapred.LocalDistributedCacheManager: Localized file:/content/mapper.py as file:/tmp/hadoop-root/mapred/local/job_local18

Check result

In [ ]:
%%bash
hdfs dfs -ls output_aggregate
hdfs dfs -cat output_aggregate/part-00000 

Found 2 items
-rw-r--r--   1 root supergroup          0 2022-10-08 17:52 output_aggregate/_SUCCESS
-rw-r--r--   1 root supergroup        326 2022-10-08 17:52 output_aggregate/part-00000
--	1
bluetooth:	2
cups:	12
ftpd	916
gdm(pam_unix)	2
gdm-binary	1
gpm	2
hcid	1
irqbalance:	1
kernel:	76
klogind	46
login(pam_unix)	2
logrotate:	43
named	16
network:	2
nfslock:	1
portmap:	1
random:	1
rc:	1
rpc.statd	1
rpcidmapd:	1
sdpd	1
snmpd	1
sshd(pam_unix)	677
su(pam_unix)	172
sysctl:	1
syslog:	2
syslogd	7
udev	8
xinetd	2


Pretty-print table of aggregated data

In [ ]:
%%bash
hdfs dfs -get output_aggregate/part-00000 result # download results file
column -t result|sort -k2nr # sort by field 2 numerically in descending order

ftpd             916
sshd(pam_unix)   677
su(pam_unix)     172
kernel:          76
klogind          46
logrotate:       43
named            16
cups:            12
udev             8
syslogd          7
bluetooth:       2
gdm(pam_unix)    2
gpm              2
login(pam_unix)  2
network:         2
syslog:          2
xinetd           2
--               1
gdm-binary       1
hcid             1
irqbalance:      1
nfslock:         1
portmap:         1
random:          1
rc:              1
rpcidmapd:       1
rpc.statd        1
sdpd             1
snmpd            1
sysctl:          1


get: `result': File exists


## Stop cluster

When you're done with your computations, you can shut down the Hadoop cluster and stop the `sshd` service.

In [ ]:
!./hadoop-3.3.4/sbin/stop-dfs.sh

Stopping namenodes on [localhost]
Stopping datanodes
Stopping secondary namenodes [93ad6cb45e12]


Stop the `sshd` daemon

In [ ]:
!/etc/init.d/ssh stop

 * Stopping OpenBSD Secure Shell server sshd
   ...done.


## Concluding remarks

We have started a single-node Hadoop cluster and ran some simple HDFS and MapReduce commands. 

Even when running on a single machine, one can benefit from the parallelism provided by multiple virtual cores.

Hadoop provides also a command-line utility (the CLI MiniCluster) to start and stop a single-node Hadoop cluster "_without the need to set any environment variables or manage configuration files_" (https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/CLIMiniCluster.html). Alas, I was not able to get the MiniCluster to run due to some uncompatible libraries if I remember correctly. 

If on one hand it would be great to be able to start a Hadoop cluster with a single command, it is interesting to see how the single components of a Hadoop cluster come into play and it helps to learn about the Hadoop architecture.

If you liked this notebook, check also:
 - [Hadoop single-node cluster setup with Python](https://github.com/groda/big_data/blob/master/Hadoop_single_node_cluster_setup_Python.ipynb) similar to this but using Python in place of bash
 - [Setting up Spark Standalone on Google Colab](https://github.com/groda/big_data/blob/master/Hadoop_Setting_up_Spark_Standalone_on_Google_Colab.ipynb)
 - [Getting to know the Spark Standalone Architecture](https://github.com/groda/big_data/blob/master/Spark_Standalone_Architecture_on_Google_Colab.ipynb)


